In [ ]:
import os
import glob
import shutil
import cv2


combinations = ['convert_to_rgb-set_dpi']
# combinations = ['set_dpi-increase_contrast-increase_sharpness']

image_path = 'C:\matsya\india_dl'
for image in glob.glob(image_path+'\\*.jpg'):
    # filename = os.path.split(image)[-1].split('.jpg')[0]
    # print(filename)
    # preproc_image = 'preprocessed_ensemble\\'+filename+'.jpg'
    # shutil.copy(image, preproc_image)
    for combination in combinations:
        filename = os.path.split(image)[-1].split('.jpg')[0]
        preproc_image = image_path+'\\preprocessed_images3\\'+filename+'.jpg'
        shutil.copy(image, preproc_image)
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = image_path+'\\preprocessed_images3\\'+filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(image_name, img)

        psm_values = [6]

        for psm in psm_values:

            parsed_output = image_path+'\\output3\\'+filename+'_'+combination+'_'+str(psm)
            command = 'tesseract {} {} -l eng --psm {}'.format(preproc_image, parsed_output, psm)
            os.system(command)

In [ ]:
import glob
import os

for file in glob.glob('C:\matsya\india_dl_preproc\\*lokesh*.txt'):
    with open(file, 'r', encoding='utf-8') as fp:
        text = fp.read()
    text = text.splitlines()
    text = [line for line in text if not line.isspace() and len(line) > 0]
    filename = os.path.split(file)[-1].split('.txt')[0]

    print(text)

In [17]:
################### ensemble the output files #####################

import glob
import os

def ensemble_output():

    for file in glob.glob('C:\matsya\india_dl_preproc\\*puneet*.txt'):
        filename = os.path.split(file)[-1].split('.txt')[0]
        ind = filename.index('dl')
        filename = filename[:ind+2]
        # print(filename)
        with open(file, encoding="utf-8") as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open('C:/matsya/india_dl_preproc/ensemble_files/{}.txt'.format(filename), 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")

ensemble_output()

In [34]:
import re
import glob
import os
from datetime import datetime

keys = {
    'dob': ['dob', 'date of birth']
}

def get_dl_number(text):
    for line in text:
        regex = r'(([A-Z]{2}[0-9]{2})( )|([A-Z]{2}-[0-9]{2}))((19|20)[0-9][0-9])[0-9]{7}$'
        dl = re.search(regex, line)
        if dl:
            return dl.group()
        
def get_name(text):
    for i in range(len(text)):
        if 'name' in text[i].lower():
            name = text[i+1]
            return name
        
def get_dob(text):
    for i in range(len(text)):
        # print([word for word in keys['dob']])
        found = any(word in text[i].lower() for word in keys['dob'])
        # print(found)
        if found:
            return text[i+1]

def get_care_of_relation_name(text):
    cor = ''
    keys = ['Son', 'Daughter', 'Wife', 'S/W/D']
    for i in range(len(text)):
        for key in keys:
            if key in text[i]:
                cor = text[i+1]
                return cor
            
def is_date(date):
    formats = ['%d-%b-%Y', '%d-%m-%Y']
    
    for date_format in formats:
        try:
            d = datetime.strptime(str(date), date_format)
            d = datetime.strftime(d, date_format)
            return d
        except ValueError:
            continue
    return False

def get_validity_date(text):
    for i in range(len(text)):
        if 'validity' in text[i].lower():
            for j in range(i, len(text)):
                d = is_date(text[j].split()[-1].strip())
                if d:
                    return d

def get_issue_date(text):
    d = ''
    vd = get_validity_date(text)
    for i in range(len(text)):
        if 'issue date' in text[i].lower():
            for j in range(i, len(text)):
                d = is_date(text[j].split()[-1].strip())
                if d and vd and d<vd:
                    return d
                
def get_blood_group(text):
    bg = ''
    pattern = '(A|B|AB|O)[+-]'
    for i in range(len(text)):
        bg = re.search(pattern, text[i])
        if bg:
            return bg.group()

def get_address(text):
    address = ''
    for i in range(len(text)):
        if 'address' in text[i].lower():
            for j in range(i+1, len(text)):
                address += text[j]+' '
                a = text[j].split()[-1]
                if len(a) == 6 and a.isnumeric() and address:
                    return address
                

for file in glob.glob('C:\pocs\dl\ensemble_files/*.txt'):
    # print(file)
    dl_number_final = ''
    name_final = ''
    dob_final = ''
    cor_final = ''
    val_date_final = ''
    issue_date_final = ''
    blood_group_final = ''
    address_final = ''

    ind = len(filename.split('\\'))-1
    with open(file) as fp:

        filename = os.path.split(file)[-1]
        words = filename.split('_')
        for i in range(len(words)):
            if 'page' in words[i]:
                ind = i
                break
        filename = '_'.join(words[:ind])
        # print(filename)

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            dl_number = get_dl_number(text)
            if dl_number and not dl_number_final:
                dl_number_final = dl_number

            name = get_name(text)
            if name and not name_final:
                name_final = name

            dob = get_dob(text)
            if dob and not dob_final:
                dob_final = dob

            cor = get_care_of_relation_name(text)
            if cor and not cor_final:
                cor_final = cor

            val_date = get_validity_date(text)
            if val_date and not val_date_final:
                val_date_final = val_date

            issue_date = get_issue_date(text)
            if issue_date and not issue_date_final:
                issue_date_final = issue_date

            blood_group = get_blood_group(text)
            if blood_group and not blood_group_final:
                blood_group_final = blood_group

            address = get_address(text)
            if address and not address_final:
                address_final = address
    
    print(dl_number_final)
    print(name_final)
    print(dob_final)
    print(cor_final)
    print(val_date_final)
    print(issue_date_final)
    print(blood_group_final)
    print(address_final)
    print()

HR26 20170006785
LOKESH YADAV
10-Sep-1998
BHUPINDRA SINGH
11-Apr-2037
12-Apr-2017
B+
282 TikK167),Gurgaon,HR 122001 

UP81 20220025702
PUNEET KUMAR GAUR
20-09-2000
SUDHIR KUMAR GAUR
19-09-2040
01-12-2022

5/86 NEAR ITI HARI VILAS NAGAR Koil, Aligarh, UP 202001 



In [31]:
s = 'DLNUMBER HR26 20170006785'
r = r'(([A-Z]{2}[0-9]{2})( )|([A-Z]{2}-[0-9]{2}))((19|20)[0-9][0-9])[0-9]{7}$'
res = re.search(r, s)
print(res.group())

HR26 20170006785


In [43]:
print('LOKESH YADAV'.strip(' '))

LOKESH YADAV


In [15]:
nums = [0,1,2,2,3,0,4,2]
val = 2
i=0
k=0
while nums[i] != '_':
    if nums[i] == val:
        nums.pop(i)
        nums.append('_')
        k += 1
    else:
        i += 1

print(nums)
print(k)

[0, 1, 3, 0, 4, '_', '_', '_']
3


In [31]:
class Solution:
    def removeElement(self, nums, val):
        i = 0
        k = 0
        if len(nums) > 0:
            while i < len(nums) and nums[i] != '_':
                if nums[i] == val:
                    nums.pop(i)
                    nums.append('_')
                else:
                    i += 1
                    k += 1
            print(nums)
            return k

In [32]:
obj = Solution()
nums = [3]
val = 2
k = obj.removeElement(nums, val)
print(k)

[3]
1


In [58]:
class Solution:
    def removeDuplicates(self, nums) -> int:
        index = 0
        k = 0
        while index < len(nums)-1:

            if nums[index] < nums[index+1]:
                index += 1
            else:
                nums.pop(index+1)

        print(nums)
        k = len(nums)
        return k

In [60]:
obj = Solution()
nums = [0,0,1,1,1,2,2,3,3,4]
obj.removeDuplicates(nums)

[0, 1, 2, 3, 4]


5

In [71]:
class Solution:
    def removeDuplicates(self, nums) -> int:
        i=0
        count = 1
        while i < len(nums)-1:
            if nums[i] == nums[i+1]:
                count += 1
                if count > 2:
                    nums.pop(i+1)
                    continue
                i += 1
            else:
                count = 1
                i += 1
        print(nums)

In [73]:
obj = Solution()
nums = [0,0,1,1,1,1,2,3,3]
obj.removeDuplicates(nums)

[0, 0, 1, 1, 2, 3, 3]


In [90]:
nums = [1]
nums.sort()

maj = len(nums)//2
print(maj)
m = 0

for i in range(len(nums)-1):
    if nums[i] == nums[i+1]:
        m += 1
        if m >= maj:
            print(nums[i])
            break
    else:
        m = 0
print(nums)

0
[1]


In [108]:
k = 6
nums = [1,2,3,4,5,6,7]
# output = [5,6,7,1,2,3,4]
l = len(nums)-1
nums2 = nums[k+1:]
nums[k+1:] = nums[:k]
nums[:k] = nums2
nums

[7, 1, 2, 3, 4, 5, 6]

In [110]:
k = 6
nums = [1,2,3,4,5,6,7]

for j in range(k):
    t = nums[-1]
    for i in range(len(nums)-2, -1, -1):
        nums[i+1] = nums[i]
    nums[0] = t

print(nums)

[2, 3, 4, 5, 6, 7, 1]


In [141]:
class Solution:
    def rotate(self, nums, k) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        nums2 = nums.copy()
        last = len(nums)-1
        for i in range(k):
            nums[0] = nums2[-1]
            nums[1:] = nums2[:last]
            nums2 = nums.copy()
        print(nums)
        

In [142]:
obj = Solution()
nums = [-1,-100,3,99]
k = 2
obj.rotate(nums, k)

[3, 99, -1, -100]


In [127]:
nums = [1,2,3,4,5,6]
nums2 = nums.copy()

nums2[0] = nums[-1]
# print(nums)
nums2[1:] = nums[:last]

# print(nums2)

[1, 2, 3, 4, 5, 6]


In [181]:
nums = [1,2,3]
k = 7
l = len(nums)

if k>len(nums):
    k=k%len(nums)
print(k)

for i in range(l//2):
    t = nums[i]
    nums[i] = nums[l-i-1]
    nums[l-i-1] = t

# print(nums)

for i in range(k//2):
    t = nums[i]
    nums[i] = nums[k-i-1]
    nums[k-i-1] = t

j = k+((l-k)//2)

m=0
for i in range(k, j):
    
    t = nums[i]
    nums[i] = nums[l-m-1]
    nums[l-m-1] = t
    m += 1

print(nums)

1
[3, 1, 2]


In [177]:
4%3

1

In [189]:
prices = [7,6,4,3,1,9]

max_profit = -1

for i in range(len(prices)-1):
    for j in range(i+1, len(prices)):
        if max_profit < prices[j] - prices[i]:
            max_profit = prices[j] - prices[i]

if max_profit == -1:
    max_profit = 0

print(max_profit)

8


In [199]:
prices = [7,1,5,3,6,4]

d = {}
for i in range(len(prices)):
    d[i] = prices[i]

sorted_prices = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))

last_k = list(sorted_prices.keys())[0]
last_v = sorted_prices[last_k]

profit = 0
for k,v in sorted_prices.items():
    if k>last_k and v>last_v:
        if profit < v-last_v:
            profit = v-last_v

print(profit)

5


In [213]:
prices = [1,2,3,4,5]

buy = prices[0]
loc_max = 0
max_profit = 0

for i in range(1, len(prices)):
    # loc_max = 0

    if prices[i] < prices[i-1]:
        buy = prices[i]
        # max_profit += loc_max
        # loc_max = 0

    elif prices[i]-buy > loc_max:
        loc_max = prices[i]-buy
        max_profit += loc_max
        # loc_max = 0
        print(loc_max)


print(max_profit)

1
2
3
4
10


In [217]:
nums = [3,2,1,0,4]
flag = 0
for i in range(len(nums)):
    if i+nums[i] == len(nums)-1:
        print(True)
        flag = 1
        break
if not flag:
    print(False)

False


In [232]:
nums = [-1, 8, 0, 0, 10, 19, 20, 0, -16]

i = 0
j = 1

while j < len(nums):
    if nums[j] == 0 and nums[i] != 0:
        j += 1
        i += 1
    elif nums[i] == 0 and nums[j] != 0:
        temp = nums[i]
        nums[i] = nums[j]
        nums[j] = temp
        i += 1
        j += 1
    elif nums[i] == 0 and nums[j] == 0:
        j += 1
    elif nums[i] != 0 and nums[j] != 0:
        i += 2
        j += 2

print(nums)

[-1, 8, 10, 19, 20, -16, 0, 0, 0]


In [230]:
s = "   fly me   to   the moon  "

s = s.strip()
words = s.split()
print(len(words[-1]))

4


In [30]:
s = '282 TikK167),Gurgaon,HR 122001'
s.isalnum()

False

In [13]:
import os

img = 'C:/matsya/india_dl/puneet_dl_original.jpg'
command = 'tesseract {} output2 -l eng --psm 4'.format(img)
print(os.system(command))

0


In [1]:
print('hello')

hello
